# Toxicity Classification with KerasNLP
## Table of Contents
* [1. Overview](#1.)
* [2. Configuration](#2.)
* [3. Setup](#3.)
* [4. Import datasets](#4.)
* [5. Data Preprocessing](#5.)
    * [5.1 Train Validation Split](#5.1)
    * [5.2 Create TensorFlow Dataset](#5.2)
* [6. Model Development](#6.)
    * [6.1 Building model](#6.1)
    * [6.2 Training model](#6.2)
    * [6.3 Evaluating model](#6.3)
* [7. Submission](#7.)
* [8. References](#8.)

<font color="red" size="3">If you found it helpful, please don't forget to upvote.</font>

<a id="1."></a>
## 1. Overview
In this notebook, I am going to build a Jigsaw Toxicity Classification Model using [DistilBERT](https://keras.io/api/keras_nlp/models/distil_bert/) from [KerasNLP Library](https://keras.io/api/keras_nlp).

DistilBERT is a distiled version of BERT which leverages Knowledge Distillation, it retrains 97% of language understanding capabilities of original BERT, while being 40% smaller and 60% faster.

KerasNLP is a Library based on Keras that makes it easier to implement NLP appplication by writing only a few lines of code. As you can see below.
```python
def get_model(config):
    encoder = keras_nlp.models.DistilBertBackbone.from_preset(
        "distil_bert_base_en_uncased"
    )
    encoder.trainable = False
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset("distil_bert_base_en_uncased")
    inputs = keras.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    x = encoder(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    model.compile(
        "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
    )
    return model
```

<a id="2."></a>
## 2. Configuration

In [1]:
class Config:
    batch_size = 128
    validation_split = 0.15
    epochs = 10 # Number of Epochs to train
    model_path = "model.tf"
    output_dataset_path = "../input/toxicity-keras-nlp-model"
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    modes = ["training", "inference"]
    mode = modes[1]
    model_name = "distil_bert_base_en_uncased"
config = Config()

<a id="3."></a>
## 3. Setup

Now install KerasNLP Library and import necessary packages.

In [2]:
pip install keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 12.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
gcsfs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.4.0 which is incompatible.
cudf 23.4.0 requires 

In [3]:
import pandas as pd
import tensorflow as tf
import pathlib
import random
import string
import re
import sys
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
import sklearn
import seaborn as sns
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import rankdata
import json
import keras_nlp

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<a id="4."></a>
## 4. Import datasets

In [4]:
!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
!unzip ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip

Archive:  ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
  inflating: train.csv               
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
  inflating: test.csv                
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
  inflating: test_labels.csv         
Archive:  ../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [5]:
train = pd.read_csv("/kaggle/working/train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


<a id="5."></a>
## 5. Data Preprocessing

<a id="5.1"></a>
### 5.1 Train Validation Split

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train["comment_text"], train[config.labels], test_size=config.validation_split)

In [7]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((135635,), (135635, 6), (23936,), (23936, 6))

<a id="5.2"></a>
### 5.2 Create TensorFlow Dataset

In [8]:
def make_dataset(X, y, batch_size, mode):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if mode == "train":
       dataset = dataset.shuffle(batch_size * 4) 
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE).repeat(1)
    return dataset

In [9]:
train_ds = make_dataset(X_train, y_train, batch_size=config.batch_size, mode="train")
valid_ds = make_dataset(X_val, y_val, batch_size=config.batch_size, mode="valid")

Let's take a look at the format of training data.

In [10]:
for batch in train_ds.take(1):
    print(batch)

(<tf.Tensor: shape=(128,), dtype=string, numpy=
array([b"and \nBenon, perhaps you could explain why are are now seeking to have me banned from editing certain pages?  My reply to your motion can be found here.  Please tell me how I have been uncivil to T-Man since the block lifted, with a link to that uncivil edit, or how I have wikistalked, since both Batman and Enemies of Batman are on  my usual edit lists without any need for me to refer to T-man's edits.  Thank you.",
       b"Why is the backward chaining example the same as the forward chaining example if they are different?\n\nThis article isn't very clear.",
       b"British BC plugs don't conform to any British Standard, which is why they vanished from the market place when conformance with British Standards became mandatory for all electrical accessories. It would have been up to one of the manufacturers to get them added to BS 5042 (the then standard for BC lampholders, now replaced by BS EN 61184), but presumably no manufact

<a id="6."></a>
## 6. Model Development

<a id="6.1"></a>
### 6.1 Building model

In [11]:
def get_model(config):
    encoder = keras_nlp.models.DistilBertBackbone.from_preset(
        config.model_name
    )
    encoder.trainable = False
    preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(
        config.model_name
    )
    inputs = keras.Input(shape=(), dtype=tf.string)
    x = preprocessor(inputs)
    x = encoder(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(6, activation="sigmoid")(x)
    model = keras.Model(inputs, output, name="model")
    model.compile(
        "adam", loss="binary_crossentropy", metrics=["categorical_accuracy", keras.metrics.AUC()]
    )
    return model

In [12]:
model = get_model(config)
model.summary()

231508/231508 [==============================] - 0s 1us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 distil_bert_preprocessor (Dist  {'token_ids': (None  0          ['input_1[0][0]']                
 ilBertPreprocessor)            , 512),                                                           
                                 'padding_mask': (N                                               
                                one, 512)}                                                        
                                                                                                  
 distil_bert_backbone (DistilBe  

<a id="6.2"></a>
### 6.2 Training model

In [13]:
if config.mode == config.modes[0]:
    checkpoint = keras.callbacks.ModelCheckpoint(config.model_path, monitor="val_categorical_accuracy", save_best_only=True)
    early_stopping = keras.callbacks.EarlyStopping(patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(patience=5, min_delta=1e-4, min_lr=1e-6)
    model.fit(train_ds, epochs=config.epochs, validation_data=valid_ds, callbacks=[checkpoint, reduce_lr])

<a id="6.3"></a>
### 6.3 Evaluating model

#### Classification Report

In [14]:
if config.mode == config.modes[0]:
    from sklearn.metrics import classification_report
    y_pred = np.array(model.predict(valid_ds) > 0.5, dtype=int)
    cls_report = classification_report(y_val, y_pred)
    print(cls_report)

<a id="7."></a>
## 7. Submission

In [15]:
test = pd.read_csv("/kaggle/working/test.csv")
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [16]:
sample_submission = pd.read_csv("/kaggle/working/sample_submission.csv")
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [17]:
test_ds = tf.data.Dataset.from_tensor_slices((test["comment_text"])).batch(config.batch_size).cache().prefetch(1)
path = config.model_path
if config.mode == config.modes[1]:
    path = config.output_dataset_path + "/" + path
model.load_weights(path)
score = model.predict(test_ds)

1197/1197 [==============================] - 1974s 2s/step


In [18]:
sample_submission[config.labels] = score
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.986544,0.292551,0.965777,0.035978,0.933436,0.273937
1,0000247867823ef7,0.016679,0.001728,0.014305,0.001217,0.007386,0.001578
2,00013b17ad220c46,0.007781,0.003409,0.008453,0.004491,0.006077,0.005879
3,00017563c3f7919a,0.000735,0.000334,0.000385,0.001475,0.000734,0.000286
4,00017695ad8997eb,0.020397,0.001156,0.010327,0.001277,0.008901,0.002058



<a id="8."></a>
## 8. References
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762v5)
- [DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter](https://arxiv.org/abs/1910.01108)
- [DistilBERT documentation](https://keras.io/api/keras_nlp/models/distil_bert/)